# Parsing bodyshops map data

In this notebook we import the [repar shops info](https://www.google.com/maps/d/u/2/viewer?hl=en&ll=-23.559310427829978%2C-46.558040275696094&z=10&mid=1T5GSP5wYUiKYY4_VLIxNI086eKYZ8q1v) and parse the information into a table

---
## 1. Parse the data


In [1]:
from pykml import parser
from zipfile import ZipFile
import re, os, psycopg2, math
import pandas as pd
import numpy as np
from tqdm import tqdm


Load the file

In [2]:
# Local filename
filename = 'data/Justos Bodyshops.kmz'

# Unzip and load the encoded kml
kmz = ZipFile(filename, 'r')
kml = kmz.open('doc.kml', 'r').read()

# Parse the file
root = parser.fromstring(kml).Document
root.descendantpaths()


['{http://www.opengis.net/kml/2.2}Document',
 '{http://www.opengis.net/kml/2.2}Document.name',
 '{http://www.opengis.net/kml/2.2}Document.description',
 '{http://www.opengis.net/kml/2.2}Document.Style',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.Icon',
 '{http://www.opengis.net/kml/2.2}Document.Style.IconStyle.Icon.href',
 '{http://www.opengis.net/kml/2.2}Document.Style.LabelStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style.LabelStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style[1]',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.scale',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.Icon',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].IconStyle.Icon.href',
 '{http://www.opengis.net/kml/2.2}Document.Style[1].LabelStyle',
 '{htt

Parse the data

In [3]:
# List of items
all_items = []

# For each entity type
for document in root.Folder[:]:
    
    # Collect the entity type
    entity_type = str(document.name)
    
    # For each placemark
    for placemark in document.Placemark[:]:
        
        placemark_name = str(placemark.name)
        
        # Try to obtain the coordinates
        try:
            placemark_coords = [float (x) for x in re.sub('[^0-9,.]', "", str(placemark.Point.coordinates)).split(',')]
        except:
            placemark_coords = ''
            
        # Try to obtain te address
        try:
            placemark_address = str(placemark.address)
        except:
            placemark_address = ''
            
        # Try to obtain the details
        try:
            location_details = {y[0]:y[1] for y in [x.split(':') for x in str(placemark.description).split('<br>')]}
        except:
            location_details = {}
            
        # Create new entry
        new_item = {'entity_type': entity_type,
                    'placemark_name': placemark_name,
                    'placemark_coords': placemark_coords,
                    'placemark_address': placemark_address}
        for k,v in location_details.items():
            new_item[k]=v if str(v).replace(' ','')!='' else np.nan
            
        # Add to list
        new_item = all_items.append(new_item)
        
        
# Create dataframe
all_items = pd.DataFrame(all_items).drop(columns='name')
all_items.replace('', np.nan, inplace=True)
all_items.head()



,entity_type,placemark_name,placemark_coords,placemark_address,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
0,Cities,Arujá,"[46.3227109, 23.3951365, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cities,PIN 01 - BEXIGA,"[46.6395571, 23.5557714, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cities,Barueri,"[46.8785468, 23.5035117, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cities,Biritiba-Mirim,"[46.0409059, 23.5704516, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cities,Caieiras,"[46.7466172, 23.362065, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for col in all_items.columns[-2:]:
    display(all_items[all_items[col].notna()])
    
all_items.drop(columns=all_items.columns[-2:], inplace=True)

,entity_type,placemark_name,placemark_coords,placemark_address,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
141,12 Pins,PRO SERVICE SERVIÇOS - Itaim,"[46.6777467, 23.5841772, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4553010 SAO PAULO - SP,NaN


,entity_type,placemark_name,placemark_coords,placemark_address,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),"AV. BANDEIRANTES, 523 VILA OLIMPIA CEP","R MAJ DIOGO, 590 BELA VISTA CEP"
142,12 Pins,RESTAUCAR - Paraiso,"[46.6427902, 23.57545, 0.0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01324000 SAO PAULO - SP


---
## 2. Sanity checks

Can't access the addresses I do see in Maps for some places, for example `Dencar` in `Repair shops (to be visited)`

In [5]:
print(root.Folder[2].Placemark[0].name)
display(root.Folder[2].Placemark[0].descendantpaths())
display(root.Folder[2].Placemark[0].Point.descendantpaths())

Dencar


['{http://www.opengis.net/kml/2.2}Placemark',
 '{http://www.opengis.net/kml/2.2}Placemark.name',
 '{http://www.opengis.net/kml/2.2}Placemark.styleUrl',
 '{http://www.opengis.net/kml/2.2}Placemark.Point',
 '{http://www.opengis.net/kml/2.2}Placemark.Point.coordinates']

['{http://www.opengis.net/kml/2.2}Point',
 '{http://www.opengis.net/kml/2.2}Point.coordinates']

Count by entity type in the map

In [6]:
all_items.entity_type.value_counts()

Cities                             43
Contrato assinado                  40
BUSCAR                             22
Repair shops (to be visited)       21
12 Pins                            16
Repair shops (waiting sign-off)     2
Name: entity_type, dtype: int64

There's some bodyshops with no description!

In [7]:
bodyshops = all_items[all_items['entity_type']=='Contrato assinado'].reset_index().drop(columns='index').copy()
bodyshops.isna().sum()

entity_type                          0
placemark_name                       3
placemark_coords                    37
placemark_address                    3
Razão Social                         3
CNPJ                                 3
Telefone                             3
Responsável                          3
Logradouro                           3
Número                               3
Complemento                         39
Bairro                               3
Cidade                               3
Estado                               3
CEP                                  3
Horário de recebimento (início)      3
Horário de recebimento (término)     3
dtype: int64

There's indeed 3 bodyshops with no name or descriptions

In [8]:
[x.name for x in root.Folder[4].Placemark[:]]

['Scattini Funilaria',
 'Oficina Marques (Vila Mascote)',
 'Oficina Marques (Santo Amaro)',
 'Ponce e Filho',
 'Dimas Norte',
 'Coronato',
 'Evolution',
 'Quatrocentos',
 'Dikar',
 'New Fenix',
 'Volens Reparações',
 'Goias Car (São Caetano)',
 'Goias Car (Santo Andre)',
 'Solução Automotiva',
 'Personal Garage',
 'Quadro Rodas Mais',
 'Torque Tamboré',
 'Solicar',
 'Centro Automotivo GL',
 'Funilaria Paulista',
 'Corifeu Automec',
 'Celio´s Car Taboão',
 'CB Car',
 'Nicola Garage',
 'Seadriccar',
 'Jotecar',
 'EcoFix ',
 'MsFix',
 'Auto Palace',
 'Club  Car',
 'Flama Car',
 'Volare Reparos',
 'ArtSystens',
 'Donicar',
 'Ribeiro Renovadora',
 'Studio Marilia Botelho',
 'Office Car',
 '',
 '',
 '']

In [9]:
[x.description for x in root.Folder[4].Placemark[:]][-3:]

['Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Bonneville',
 'Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Car Solution Express',
 'Razão Social: <br>CNPJ: <br>Telefone: <br>Responsável: <br>Logradouro: <br>Número: <br>Complemento: <br>Bairro: <br>Cidade: <br>Estado: <br>CEP: <br>Horário de recebimento (início): <br>Horário de recebimento (término): <br>name: Mec Max']

---
### 3. Assign latitude and longitude to addresses

We work with current bodyshops. Download table of coords and CEP codes

In [10]:
# Create the geolocator object
from tqdm import tqdm
from geopy.geocoders import MapBox
geolocator = MapBox(api_key=os.environ['MAPBOX_DS_TOKEN']# timeout=time_out, adapter_factory=AioHTTPAdapter,
           )

# Function to modify the endpoint to places-permanent which apparently let's you export the results (I don't need it)
def places_permanent(geolocator):
    geolocator.api_path = "/geocoding/v5/mapbox.places-permanent/{query}.json/"
    geolocator.api = "%s://%s%s" % (
        geolocator.scheme,
        geolocator.domain,
        geolocator.api_path,
    )
    return geolocator

# Create the geolocator object
from geopy.geocoders import MapBox
geolocator = MapBox(api_key=os.environ['MAPBOX_DS_TOKEN']# timeout=time_out, adapter_factory=AioHTTPAdapter,
           )

# Now for each bodyshop
responses_formatted_address = []
responses_raw_address = []
for i, row in tqdm(bodyshops.iterrows()):
    
    ## QUERY FOR A FORMATTED ADDRESS
    try:

        # Create the address for the request
        request_address = row['Logradouro'].lstrip() + ", " +\
                          row['Número'].lstrip() + ", " +\
                          row['Cidade'].lstrip() + ", " +\
                          row['Bairro'].lstrip() + " - " +\
                          row['Estado'].lstrip() + ", " +\
                          row['CEP'].lstrip() + ", Brazil"

        # Send the request
        response = geolocator.geocode(request_address,
                                      exactly_one=False,
                                      country='BR')

        # Store the output
        responses_formatted_address.append([{'mapped_address': result.address,
                                             'latitude': result.latitude,
                                             'longitude': result.longitude} for result in response])

    except:
        responses_formatted_address.append({'mapped_address': np.nan,
                                            'latitude': np.nan,
                                            'longitude': np.nan})
        
        
    ## QUERY FOR THE RAW STORED ADDRESS   
    if row['placemark_address'] not in [np.nan, '']:
        
        # Send the request
        response = geolocator.geocode(row['placemark_address'],
                                      exactly_one=False,
                                      country='BR')

        # Store the output
        responses_raw_address.append([{'mapped_address': result.address,
                                       'latitude': result.latitude,
                                       'longitude': result.longitude} for result in response])

    else:
        responses_raw_address.append({'mapped_address': np.nan,
                                      'latitude': np.nan,
                                      'longitude': np.nan})
    

40it [00:06,  5.97it/s]


Review results to assign the correct one - typically the best result shares CEP, but many times this isn't the case... at some point I'd have to automate this

In [11]:
correct_result_index = {
    0: 0,
    1: 0,
    2: 1,
    3: 1,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: -1,
    9: -1,
    10: 0,
    11: 0,
    12: 0,
    13: -1,
    14: -1,
    15: 0,
    16: -1,
    17: 0,
    18: -1,
    19: 3,
    20: 2,
    21: -1,
    22: 0,
    23: -1,
    24: -1,
    25: 2,
    26: -1,
    27: 1,
    28: 0,
    29: 2,
    30: 1,
    31: -1,
    32: -1,
    33: 0,
    34: 0,
    35: 0,
    36: -1,
    37: -1,
    38: -1,
    39: -1
}

Addresses that didn't work out

In [12]:
for k,v in correct_result_index.items():
    if v==-1:
        display(responses_formatted_address[k])
        print()
        #display(responses_raw_address[k])
        #print()
        for key in ['placemark_address', 'Logradouro', 'Número', 'Bairro', 'Cidade', 'CEP']:
            print(key)
            print(bodyshops.iloc[k][key])
        print('\n----\n')
    

[{'mapped_address': 'Rudge Ramos, São Bernardo Do Campo - São Paulo, Brazil',
  'latitude': -23.6591,
  'longitude': -46.5683},
 {'mapped_address': 'Hocca Bar, São Bernardo Plaza Shopping, São Bernardo Do Campo, São Paulo 09790, Brazil',
  'latitude': -23.722901,
  'longitude': -46.543064},
 {'mapped_address': 'Cantina do Zelão, R. Jurubatuba, 926, São Bernardo Do Campo, São Paulo 09725, Brazil',
  'latitude': -23.705457,
  'longitude': -46.553238},
 {'mapped_address': 'Giramundo Bar & Ristorante, R. Olegário Herculano, 235, São Bernardo Do Campo, São Paulo 09732, Brazil',
  'latitude': -23.682619,
  'longitude': -46.55846},
 {'mapped_address': 'Padaria Bela Petrópolis, R. Cidade de São Sebastião, 203, São Bernardo Do Campo, São Paulo 09761, Brazil',
  'latitude': -23.710201,
  'longitude': -46.53935}]


placemark_address
R. Marabá 80 São Bernardo do Campo 09635-040
Logradouro
 R. Marabá
Número
 80
Bairro
 Rudge Ramos
Cidade
 São Bernardo do Campo
CEP
 09635-040

----



[{'mapped_address': 'Rua Amador Bueno, Piratininga, Osasco - São Paulo, 06236, Brazil',
  'latitude': -23.518342,
  'longitude': -46.794545},
 {'mapped_address': 'Rua Amador Bueno 430, Paquetá, Santos - São Paulo, 11013, Brazil',
  'latitude': -23.937599,
  'longitude': -46.319753},
 {'mapped_address': 'Rua Amador Bueno 430, Itapevi - São Paulo, 06680, Brazil',
  'latitude': -23.531616,
  'longitude': -46.977936},
 {'mapped_address': 'Rua Amador Bueno 430, Nasser, Campo Grande - Mato Grosso do Sul, 70116, Brazil',
  'latitude': -20.425939,
  'longitude': -54.635548},
 {'mapped_address': 'Rua Amador Bueno 430, Tabatinga, Camaragibe - Pernambuco, 54754, Brazil',
  'latitude': -7.991673,
  'longitude': -34.974019}]


placemark_address
R. Amador Bueno 430 Osasco 06230-100
Logradouro
 R. Amador Bueno
Número
 430
Bairro
 Piratininga
Cidade
 Osasco
CEP
 06230-100

----



[{'mapped_address': 'Rua Salvador Peluso Basile Taboão Da Serra - São Paulo, 05544, Brazil',
  'latitude': -23.59965,
  'longitude': -46.761256},
 {'mapped_address': 'Taboão Da Serra, São Paulo, Brazil',
  'latitude': -23.6122,
  'longitude': -46.7791},
 {'mapped_address': 'Santa Rosa, Rio Grande do Sul, Brazil',
  'latitude': -27.8644,
  'longitude': -54.4779},
 {'mapped_address': 'Santa Rosa, Niterói - Rio de Janeiro, Brazil',
  'latitude': -22.8992,
  'longitude': -43.0986},
 {'mapped_address': 'Santa Rosa, Cubatão - São Paulo, Brazil',
  'latitude': -23.89,
  'longitude': -46.43}]

...


placemark_address
nan
Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



{'mapped_address': nan, 'latitude': nan, 'longitude': nan}


placemark_address
nan
Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



{'mapped_address': nan, 'latitude': nan, 'longitude': nan}


placemark_address
nan
Logradouro
nan
Número
nan
Bairro
nan
Cidade
nan
CEP
nan

----



Add the correct result to the dataset

In [13]:
mapped_info = []
for k,v in correct_result_index.items():
    if v!=-1:
        mapped_info.append(responses_formatted_address[k][v])
    else:
        mapped_info.append({'mapped_address': np.nan,
                            'latitude': np.nan,
                            'longitude': np.nan})
        
mapped_info = pd.DataFrame(mapped_info)
bodyshops = pd.concat([bodyshops, mapped_info], axis=1)
display(bodyshops.head())


,entity_type,placemark_name,placemark_coords,placemark_address,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Horário de recebimento (início),Horário de recebimento (término),mapped_address,latitude,longitude
0,Contrato assinado,Scattini Funilaria,NaN,"Al,. dos Nhambiquaras 1542 São Paulo 04090-003",Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,NaN,Moema,São Paulo,SP,04090-003,8,6,"Alameda Dos Nhambiquaras 1542, São Paulo - São...",-23.613824,-46.665755
1,Contrato assinado,Oficina Marques (Vila Mascote),NaN,Av. Santa Catarina 2580 São Paulo 04378-200,Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,NaN,Vila Mascote,São Paulo,SP,04378-200,8,5,"Avenida Santa Catarina 2580, São Paulo - São P...",-23.655885,-46.655282
2,Contrato assinado,Oficina Marques (Santo Amaro),NaN,Av. Santo Amaro 4408 São Paulo 04556-500,Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,NaN,Brooklin,São Paulo,SP,04556-500,8,5,"Avenida Santo Amaro 4408, São Paulo - São Paul...",-23.621760,-46.684211
3,Contrato assinado,Ponce e Filho,NaN,R. Butiá 42 São Paulo 03346-010,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,NaN,Vila Reg. Feijó,São Paulo,SP,03346-010,8,5,"Rua Butia 42, São Paulo - São Paulo, 03346, Br...",-23.558920,-46.570322
4,Contrato assinado,Dimas Norte,NaN,Av. Gen. Ataliba Leonel 1727 São Paulo 02033-010,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,NaN,Carandiru,São Paulo,SP,02033-010,8,5,"Avenida General Ataliba Leonel 1727, São Paulo...",-23.500517,-46.613110


Tranform pin coordinates (for 3 avaiable locations) to landmark coordinates

In [14]:
bodyshops['latitude']  = bodyshops.apply(lambda row: row['latitude' ] if type(row['placemark_coords'])!=list
                                                    else -row['placemark_coords'][1], axis=1)
bodyshops['longitude'] = bodyshops.apply(lambda row: row['longitude'] if type(row['placemark_coords'])!=list
                                                    else -row['placemark_coords'][0], axis=1)
bodyshops.latitude


0    -23.613824
1    -23.655885
2    -23.621760
3    -23.558920
4    -23.500517
5    -23.512051
6    -23.523587
7    -23.673614
8           NaN
9           NaN
10   -23.689742
11   -23.616538
12   -23.664454
13          NaN
14          NaN
15   -23.464829
16          NaN
17   -23.560231
18          NaN
19   -23.527647
20   -23.573588
21          NaN
22   -23.481332
23          NaN
24          NaN
25   -23.632536
26          NaN
27   -23.536358
28   -23.581047
29   -23.563902
30   -23.556956
31          NaN
32          NaN
33   -23.497838
34   -23.563188
35   -23.395262
36          NaN
37   -23.667179
38   -23.389730
39   -23.539489
Name: latitude, dtype: float64

Sort columns and transform column names

In [16]:
bodyshops = bodyshops[['entity_type', 'placemark_name', 'placemark_address', 
                       'mapped_address', 'latitude', 'longitude'] + list(bodyshops.columns[4:17])]
bodyshops.rename(columns = {'Razão Social': 'razao_social',
                            'CNPJ': 'cnpj',
                            'Telefone': 'telefone',
                            'Responsável': 'responsavel',
                            'Logradouro': 'logradouro',
                            'Número': 'numero',
                            'Complemento': 'complemento',
                            'Bairro': 'bairro',
                            'Cidade': 'cidade',
                            'Estado': 'estado',
                            'CEP': 'cep',
                            'Horário de recebimento (início)': 'inicio_recebimento',
                            'Horário de recebimento (término)': 'termino_recebimento'},
                inplace=True)

bodyshops.head()


,entity_type,placemark_name,placemark_address,mapped_address,latitude,longitude,razao_social,cnpj,telefone,responsavel,logradouro,numero,complemento,bairro,cidade,estado,cep,inicio_recebimento,termino_recebimento
0,Contrato assinado,Scattini Funilaria,"Al,. dos Nhambiquaras 1542 São Paulo 04090-003","Alameda Dos Nhambiquaras 1542, São Paulo - São...",-23.613824,-46.665755,Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,NaN,Moema,São Paulo,SP,04090-003,8,6
1,Contrato assinado,Oficina Marques (Vila Mascote),Av. Santa Catarina 2580 São Paulo 04378-200,"Avenida Santa Catarina 2580, São Paulo - São P...",-23.655885,-46.655282,Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,NaN,Vila Mascote,São Paulo,SP,04378-200,8,5
2,Contrato assinado,Oficina Marques (Santo Amaro),Av. Santo Amaro 4408 São Paulo 04556-500,"Avenida Santo Amaro 4408, São Paulo - São Paul...",-23.621760,-46.684211,Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,NaN,Brooklin,São Paulo,SP,04556-500,8,5
3,Contrato assinado,Ponce e Filho,R. Butiá 42 São Paulo 03346-010,"Rua Butia 42, São Paulo - São Paulo, 03346, Br...",-23.558920,-46.570322,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,NaN,Vila Reg. Feijó,São Paulo,SP,03346-010,8,5
4,Contrato assinado,Dimas Norte,Av. Gen. Ataliba Leonel 1727 São Paulo 02033-010,"Avenida General Ataliba Leonel 1727, São Paulo...",-23.500517,-46.613110,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,NaN,Carandiru,São Paulo,SP,02033-010,8,5


In [17]:
for i,row in bodyshops.iterrows():
    print(row['placemark_address'])
    print(row['mapped_address'])
    print()
    

Al,. dos Nhambiquaras 1542 São Paulo 04090-003
Alameda Dos Nhambiquaras 1542, São Paulo - São Paulo, 04090, Brazil

Av. Santa Catarina 2580 São Paulo 04378-200
Avenida Santa Catarina 2580, São Paulo - São Paulo, 04378, Brazil

Av. Santo Amaro 4408 São Paulo 04556-500
Avenida Santo Amaro 4408, São Paulo - São Paulo, 04556, Brazil

R. Butiá 42 São Paulo 03346-010
Rua Butia 42, São Paulo - São Paulo, 03346, Brazil

Av. Gen. Ataliba Leonel 1727 São Paulo 02033-010
Avenida General Ataliba Leonel 1727, São Paulo - São Paulo, 02033, Brazil

Av. Santa Marina 1520 São Paulo 05036-001
Avenida Santa Marina 1520, São Paulo - São Paulo, 05036, Brazil

R. dos Italianos 708 São Paulo 01131-000
Rua Dos Italianos 708, São Paulo - São Paulo, 01131, Brazil

R. dos Inocentes 537 São Paulo 04764-050
Rua Dos Inocentes 537, São Paulo - São Paulo, 04764, Brazil

R. Marabá 80 São Bernardo do Campo 09635-040
nan

R. Amador Bueno 430 Osasco 06230-100
nan

R. Bandeirantes 362 Diadema 09912-230
Rua Bandeirantes, C

Export

In [18]:
bodyshops.to_csv('data/bodyshops.csv', index=False)